In [1]:
import pandas as pd
import numpy as np
import sys
from tsfresh.feature_extraction import extract_features, EfficientFCParameters
print(sys.version)

KeyboardInterrupt: 

##### 选择数据

In [ ]:
n="★阴性整合.csv"
n="alln++.csv"
p="★阳性整合.csv"
# p='纯冰毒-重复.csv'
# p='纯海洛因-重复.csv'
# p='混吸.csv'

In [ ]:

#读取数据
path='E:\\desktop\\data\\'
n_data = pd.read_csv(path+'pure\\[去异常]'+n,encoding="gbk")
n_name=pd.DataFrame(n_data['name'],columns=["name"])  ##提取name
n_temp=pd.DataFrame(n_data.iloc[:,1:127]) #提取数据 

p_data = pd.read_csv(path+'pure\\[去异常]'+p,encoding="gbk")
p_name=pd.DataFrame(p_data['name'],columns=["name"])  ##提取name
p_temp=pd.DataFrame(p_data.iloc[:,1:127]) #提取数据 

x_dic={
    "id":[],
    "time":[],
    "value":[],
}


for i in range(len(n_temp)):
    value=list(n_temp.iloc[i][:])

    key=i+1
    keys=[]
    for j in range(len(value)):
        keys.append(key) 

    time=[j for j in range(len(value))]

    x_dic['id']=x_dic['id']+keys
    x_dic['time']=x_dic['time']+time
    x_dic['value']=x_dic['value']+value

for i in range(len(p_temp)):
    value=list(p_temp.iloc[i][:])

    key=len(n_temp)+i+1
    keys=[]
    for j in range(len(value)):
        keys.append(key) 

    time=[j for j in range(len(value))]

    x_dic['id']=x_dic['id']+keys
    x_dic['time']=x_dic['time']+time
    x_dic['value']=x_dic['value']+value


timeseries = pd.DataFrame(x_dic)

##### 特征拓展

In [ ]:
settings = EfficientFCParameters()
fea_data = extract_features(timeseries, column_id="id", column_sort="time",default_fc_parameters=settings)
fea_list=list(fea_data.columns)
fea_data.shape

Feature Extraction:   0%|          | 0/15 [00:03<?, ?it/s]


TypeError: acf() got an unexpected keyword argument 'unbiased'

In [ ]:
def gap(n_fea,p_fea,fea_list):
    gap_dict={}
    for fea in fea_list:
        n_col=list(n_fea[fea])
        p_col=list(p_fea[fea])

        gap_list=[]
        gap_up=0
        gap_down=0
        for a in range(len(n_col)):
            gaps=[]
            for b in range(len(p_col)):
                gap=p_col[b]-n_col[a]
                gaps.append(gap)
                if gap>0:
                    gap_up=gap_up+1
                elif gap<0:
                    gap_down=gap_down+1
            gap_list.append(gaps)

        gap_bili=0
        if (gap_up+gap_down)!=0:
            if gap_up/(gap_up+gap_down)>0.5:
                gap_bili=gap_up/(gap_up+gap_down)
            else:
                gap_bili=gap_down/(gap_up+gap_down)

        gap_dict[fea]=round(gap_bili*100,2)

    gap_dict = dict(sorted(gap_dict.items(), key=lambda x: x[1],reverse=True))
    return gap_dict

##### 特征筛选

In [ ]:
n_fea_data=fea_data.iloc[:len(n_data)]
p_fea_data=fea_data.iloc[len(n_data):]
gap_dict=gap(n_fea_data,p_fea_data,fea_list)

print(len(fea_list))

delset=set()

for fea in fea_list:
    col=fea_data[fea]

    if col.std()==0 or col.std()==np.nan or gap_dict[fea]<51 or col.isnull().any()==True or sum(col)==0 or sum(col)==np.nan:
        delset.add(fea)

fea_list=list(set(fea_list)-delset)
print(len(fea_list))


n_fea_data=n_fea_data[fea_list]
p_fea_data=p_fea_data[fea_list]
n_fea_data.reset_index(drop=True,inplace=True)
p_fea_data.reset_index(drop=True,inplace=True)
n_fea_data=pd.concat([n_name,n_fea_data],axis=1)
p_fea_data=pd.concat([p_name,p_fea_data],axis=1)
gap_dict=gap(n_fea_data,p_fea_data,fea_list)
gap_dict

781
546


{'value__large_standard_deviation__r_0.25': 100.0,
 'value__ratio_beyond_r_sigma__r_3': 100.0,
 'value__variance_larger_than_standard_deviation': 100.0,
 'value__fourier_entropy__bins_2': 89.7,
 'value__last_location_of_minimum': 82.16,
 'value__fft_coefficient__attr_"imag"__coeff_3': 82.0,
 'value__agg_linear_trend__attr_"rvalue"__chunk_len_50__f_agg_"min"': 81.48,
 'value__fft_coefficient__attr_"real"__coeff_2': 81.48,
 'value__fft_coefficient__attr_"angle"__coeff_3': 81.36,
 'value__first_location_of_minimum': 81.1,
 'value__fft_coefficient__attr_"angle"__coeff_2': 80.82,
 'value__agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"mean"': 80.78,
 'value__energy_ratio_by_chunks__num_segments_10__segment_focus_2': 79.5,
 'value__autocorrelation__lag_6': 78.22,
 'value__autocorrelation__lag_5': 78.2,
 'value__autocorrelation__lag_7': 77.98,
 'value__agg_linear_trend__attr_"stderr"__chunk_len_50__f_agg_"min"': 77.8,
 'value__autocorrelation__lag_4': 77.77,
 'value__autocorrelation__l

##### 保存

In [ ]:
n_fea_data.to_csv(path+'fea\\'+'[特征]'+n,index=False,encoding='gbk')
p_fea_data.to_csv(path+'fea\\'+'[特征]'+p,index=False,encoding='gbk')

print("ts特征提取完毕 👌")


ts特征提取完毕 👌
